Lets start by defining a sigmoid function

In [ ]:
import pandas as pd
from scipy import optimize as op
import numpy as np
from math import exp

def Sigmoid(z):
    return 1/(1 + np.exp(-z))

Where z is the linear regression hypothesis given as k0 + k1 * x1 + k2 * x2. We shall be retaining k0 = 1 as a standard constant.Lets now import data and modify it to carter our needs.

In [ ]:
dataset = pd.read_csv("/Users/adityaphase/Desktop/ex2data2.csv", usecols=['X1', 'X2', 'Y1'])
x = dataset.iloc[:, [0, 1]]
y = dataset.iloc[:, [2]]
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.25, random_state = 0)
from sklearn.preprocessing import StandardScaler
sx = StandardScaler()
xtrain = sx.fit_transform(xtrain)
xtest = sx.transform(xtest)

We shall use train_test_split method to split our dataset as 75% training data and rest as test data. For larger datasets it would be a better idea to split it as 50 - 50%. The purpose of standard scaler is to run feature scaling on our input data which would help to normalise data within a particular range. Now lets convert these matrices into numpy arrays to make them easier to work with.

In [ ]:
X = np.asarray(xtrain)
y = np.asarray(ytrain)
X1 = np.asarray(xtest)
y1 = np.asarray(ytest)
y2 = []

The empty list y2 shall be used later. Lets now extract the shape of array X and initialise the initial theta as a array of 0. The size of this array will be equal to number of columns in x ( as our hypothesis is k0 + k1 * x1 + k2 * x2 ).

In [ ]:
m , n = X.shape
initialTheta = np.zeros(n)

Now lets define a cost function. Cost function minimization is crucial for finding accurate hypothesis parameters.

In [ ]:
def cost(theta,x,y):
    m,n = x.shape 
    theta = theta.reshape((n,1))
    y = y.reshape((m,1))
    n1 = (np.log(Sigmoid(x.dot(theta)))).reshape((m, 1))
    n2 = (np.log(1-Sigmoid(x.dot(theta)))).reshape((m,1))
    func = y * n1 + (1 - y) * n2
    J = -((np.sum(func))/m)
    return J

Here we have divided the two terms y(log(h(x))) and the term (1 - y)log(1 - h(x)). Doing a dot product on these vectors and subtracting them yields the final cost function. Now our main purpose is to minimise the cost function since cost function tells us how wrong is our model in predicting the relation between x and y and thus its no brainer to minimise it to achieve max accuracy. We shall be minimising the cost function using gradient descent to achieve a minima.

In [ ]:
def gradientDescent(theta,x,y):
    m , n = x.shape
    theta = theta.reshape((n,1))
    y = y.reshape((m,1))
    n1 = Sigmoid(x.dot(theta))
    grad = ((x.T).dot(n1-y))/m
    return grad.flatten()

So we define a gradientDescent function to minimise the cost. Gradient descent is in layman terms the partial derivative of the cost function coming out as sum{ (h(Xi) - Yi)Xi ) }.The flatten method is used to convert a multidimensional array to a single 1D array.

In [ ]:
Result = op.minimize(fun = cost, x0 = initialTheta, args = (X, y), method = 'TNC', jac = gradientDescent)
modelledTheta = Result.x
print(modelledTheta)

We now use the secret sauce for predicting the accurate values of theta. The minimise method is a powerful method which Newtons Conjugate Gradient to predict theta. 

In [ ]:
def predict(X1, y1, modelledTheta):
    c = 0
    for i in range(0, len(X1)):
        z = 1 + modelledTheta[0]*X1[i, 0] + modelledTheta[1]*X1[i, 1]
        y2.append(round(1 / (1 + exp(-z))))
    for i in range(0, len(y1)):
        if y2[i] == y1[i]:
            c += 1
    print("Accuracy: ", (c / len(X1)))

Time to use our predicted theta to test on our test dataset. We shall also find the accuracy of our model. Running our model on training data yields a accuracy of 92 % which is not too shabby.